In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from itertools import chain, combinations

In [2]:
# Загрузим таблицу с исходными данными
data = pd.read_csv('movie_bd_v5.csv')

# Установим количество отображаемых десятичных знаков
pd.set_option('precision', 2)
pd.options.display.float_format = '{:.2f}'.format
 
# Посмотрим на выборку случайных записей из исходной таблицы
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1110,tt0311113,150000000,212011111,Master and Commander: The Far Side of the World,Russell Crowe|Paul Bettany|James D'Arcy|Billy ...,Peter Weir,The Courage to Do the Impossible Lies in the H...,After an abrupt and violent encounter with a F...,138,Adventure,Universal Pictures|Twentieth Century Fox Film ...,11/14/2003,6.60,2003
1518,tt0493430,11000000,72778712,Jackass Number Two,Johnny Knoxville|Bam Margera|Steve-O|Chris Pon...,Jeff Tremaine,Guess who just made number two?,Jackass Number Two is a compilation of various...,95,Action|Comedy|Documentary,Paramount Pictures,9/22/2006,6.40,2006
1149,tt0309530,35000000,20298207,Down with Love,RenÃ©e Zellweger|Ewan McGregor|Sarah Paulson|D...,Peyton Reed,The ultimate catch has met his match,"Aaaah... it's New York City in 1962, and love ...",101,Action|Comedy|Romance|Thriller,Regency Enterprises|Fox 2000 Pictures|Epsilon ...,5/8/2003,6.00,2003
973,tt1853728,100000000,425368238,Django Unchained,Jamie Foxx|Christoph Waltz|Leonardo DiCaprio|K...,Quentin Tarantino,"Life, liberty and the pursuit of vengeance.","With the help of a German bounty hunter, a fre...",165,Drama|Western,Columbia Pictures|The Weinstein Company,12/25/2012,7.70,2012
1683,tt0259324,110000000,228738393,Ghost Rider,Nicolas Cage|Eva Mendes|Wes Bentley|Donal Logu...,Mark Steven Johnson,Hell Is About To Be Unleashed,"In order to save his dying father, young stunt...",114,Thriller|Action|Fantasy|Horror,Columbia Pictures Corporation|Relativity Media...,2/16/2007,5.20,2007


In [3]:
# Посмотрим сводку числовых значений
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1889.00,1889.00,1889.00,1889.00,1889.00
mean,54310829.64,155365321.08,109.66,6.14,2007.86
std,48587207.45,214669813.15,18.02,0.76,4.47
min,5000000.00,2033165.00,63.00,3.30,2000.00
25%,20000000.00,34560577.00,97.00,5.60,2004.00
50%,38000000.00,83615414.00,107.00,6.10,2008.00
75%,72000000.00,178262620.00,120.00,6.60,2012.00
max,380000000.00,2781505847.00,214.00,8.10,2015.00


In [4]:
# Изучим сводку атрибутов
data.describe(include=object)

,imdb_id,original_title,cast,director,tagline,overview,genres,production_companies,release_date
count,1889,1889,1889,1889,1889,1889,1889,1889,1889
unique,1889,1887,1878,957,1880,1889,652,1624,1435
top,tt0192255,The Gift,Jennifer Lawrence|Josh Hutcherson|Liam Hemswor...,Steven Soderbergh,The only way out is down.,Thor fights to restore order across the cosmos...,Comedy,New Line Cinema,12/25/2014
freq,1,2,3,13,2,1,141,21,7


In [5]:
# Посмотрим информацию о структуре и типах данных
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   cast                  1889 non-null   object 
 5   director              1889 non-null   object 
 6   tagline               1889 non-null   object 
 7   overview              1889 non-null   object 
 8   runtime               1889 non-null   int64  
 9   genres                1889 non-null   object 
 10  production_companies  1889 non-null   object 
 11  release_date          1889 non-null   object 
 12  vote_average          1889 non-null   float64
 13  release_year          1889 non-null   int64  
dtypes: float64(1), int64(4), object(9)
memory usage: 206.7+ KB


In [6]:
# Проверим наличие NaN
data.isna().sum()

imdb_id                 0
budget                  0
revenue                 0
original_title          0
cast                    0
director                0
tagline                 0
overview                0
runtime                 0
genres                  0
production_companies    0
release_date            0
vote_average            0
release_year            0
dtype: int64

# Предобработка

In [7]:
# Создадим словарь для ответов
answers = {}

# Рассчитаем прибыль
data['profit'] = data.revenue - data.budget

# Переведем перечень жанров из строки в список
data['list_genres']=data.genres.apply(lambda x:x.split('|'))

# Переведем перечень режиссеров из строки в список
data['list_director']=data.director.apply(lambda x:x.split('|'))

# Переведем перечень актеров из строки в список
data['list_cast']=data.cast.apply(lambda x:x.split('|'))

# Переведем перечень компаний из строки в список
data['list_production_companies']=data.production_companies.apply(lambda x:x.split('|'))

# Добавим месяц релиза
data['release_month'] = pd.to_datetime(data.release_date, format='%m/%d/%Y').dt.month

# Добавим время года релиза
SEASONS = {1: 'winter', 2: 'winter', 3: 'spring', 4: 'spring', 5: 'spring', 6: 'summer',
           7: 'summer', 8: 'summer', 9: 'autumn', 10: 'autumn', 11: 'autumn', 12: 'winter'}
data['release_season'] = data['release_month'].replace(SEASONS)

# Посмотрим на 3 случайные записи
data.sample(3)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,...,release_date,vote_average,release_year,profit,list_genres,list_director,list_cast,list_production_companies,release_month,release_season
1767,tt0419887,20000000,73276047,The Kite Runner,Khalid Abdalla|Atossa Leoni|Shaun Toub|Zekeria...,Marc Forster,There is a way to be good again.,The Kite Runner is a 2007 American drama film ...,128,Drama,...,10/14/2007,7.30,2007,53276047,[Drama],[Marc Forster],"[Khalid Abdalla, Atossa Leoni, Shaun Toub, Zek...","[DreamWorks SKG, MacDonald/Parkes Productions,...",10,autumn
1650,tt0377062,45000000,21009180,Flight of the Phoenix,Dennis Quaid|Tyrese Gibson|Giovanni Ribisi|Mir...,John Moore,"Out of the ashes, hope will rise.",When an Amacor oil rig in the Gobi Desert of M...,113,Action|Adventure|Drama|Thriller,...,12/17/2004,5.40,2004,-23990820,"[Action, Adventure, Drama, Thriller]",[John Moore],"[Dennis Quaid, Tyrese Gibson, Giovanni Ribisi,...","[Twentieth Century Fox Film Corporation, Davis...",12,winter
1464,tt0424095,149000000,64459316,Flushed Away,Hugh Jackman|Kate Winslet|Ian McKellen|Jean Re...,David Bowers|Sam Fell,Someone's Going Down,"London high-society mouse, Roddy is flushed do...",85,Adventure|Animation|Comedy|Family,...,10/22/2006,6.00,2006,-84540684,"[Adventure, Animation, Comedy, Family]","[David Bowers, Sam Fell]","[Hugh Jackman, Kate Winslet, Ian McKellen, Jea...","[Aardman Animations, DreamWorks Animation]",10,autumn


# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [8]:
answers[1] = data[data.budget==data.budget.max()].original_title.item()
answers[1]

'Pirates of the Caribbean: On Stranger Tides'

ВАРИАНТ 2

In [9]:
data.sort_values(by='budget', ascending=False)[['original_title', 'budget']].original_title.iloc[0]

'Pirates of the Caribbean: On Stranger Tides'

In [10]:
# Посмотрим топ5 фильмов с максимальным бюджетом
data.sort_values(by='budget', ascending=False)[['original_title', 'budget']].head(5)

,original_title,budget
723,Pirates of the Caribbean: On Stranger Tides,380000000
1669,Pirates of the Caribbean: At World's End,300000000
14,Avengers: Age of Ultron,280000000
1014,John Carter,260000000
380,Tangled,260000000


# 2. Какой из фильмов самый длительный (в минутах)?

In [11]:
answers[2] = data[data.runtime==data.runtime.max()].original_title.item()
answers[2]

'Gods and Generals'

In [12]:
# Посмотрим топ5 самых длительных фильмов
data.sort_values(by='runtime', ascending=False)[['original_title', 'runtime']].head(5)

,original_title,runtime
1157,Gods and Generals,214
1081,The Lord of the Rings: The Return of the King,201
1736,Grindhouse,191
1336,King Kong,187
504,Pearl Harbor,183


# 3. Какой из фильмов самый короткий (в минутах)?

In [13]:
answers[3] = data[data.runtime==data.runtime.min()].original_title.item()
answers[3]

'Winnie the Pooh'

In [14]:
# Посмотрим топ5 самых коротких по дительности фильмов
data.sort_values(by='runtime')[['original_title', 'runtime']].head(5)

,original_title,runtime
768,Winnie the Pooh,63
930,Return to Never Land,72
1097,The Jungle Book 2,72
1408,Valiant,76
1626,Home on the Range,76


# 4. Какова средняя длительность фильмов?

In [15]:
answers[4] = round(data.runtime.mean())
answers[4]

110

# 5. Каково медианное значение длительности фильмов? 

In [16]:
answers[5] = data.runtime.median()
answers[5]

107.0

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [17]:
# Код получения столбца profit вынесен в Предобработку
answers[6] = data[data.profit==data.profit.max()].original_title.item()
answers[6]

'Avatar'

In [18]:
# Посмотрим топ5 самых прибыльных фильмов
data.sort_values(by='profit', ascending=False)[['original_title', 'profit']].head(5)

,original_title,profit
239,Avatar,2544505847
3,Star Wars: The Force Awakens,1868178225
0,Jurassic World,1363528810
4,Furious 7,1316249360
970,The Avengers,1299557910


# 7. Какой фильм самый убыточный? 

In [19]:
answers[7] = data[data.profit==data.profit.min()].original_title.item()
answers[7]

'The Lone Ranger'

In [20]:
# Посмотрим топ3 самых убыточных фильмов
data.sort_values(by='profit')[['original_title', 'profit']].head(5)

,original_title,profit
1245,The Lone Ranger,-165710090
1616,The Alamo,-119180039
809,Mars Needs Moms,-111007242
958,The Adventures of Pluto Nash,-92896027
1464,Flushed Away,-84540684


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [21]:
answers[8] = sum(data.profit>0)
answers[8]

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [22]:
data_select = data[data.release_year==2008]
answers[9] = data_select[data_select.revenue==data_select.revenue.max()].original_title.item()
answers[9]

'The Dark Knight'

In [23]:
# Посмотрим самые кассовые фильмы за последние 10 лет
data.sort_values(by='revenue', ascending=False) \
    .drop_duplicates(['release_year']) \
    .sort_values(by='release_year', ascending=False) \
    [['release_year', 'original_title', 'revenue']].head(10)

,release_year,original_title,revenue
3,2015,Star Wars: The Force Awakens,2068178225
123,2014,The Hobbit: The Battle of the Five Armies,955119788
1177,2013,Frozen,1274219009
970,2012,The Avengers,1519557910
722,2011,Harry Potter and the Deathly Hallows: Part 2,1327817822
381,2010,Toy Story 3,1063171911
239,2009,Avatar,2781505847
599,2008,The Dark Knight,1001921825
1669,2007,Pirates of the Caribbean: At World's End,961000000
1434,2006,Pirates of the Caribbean: Dead Man's Chest,1065659812


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [24]:
data_select = data[(data.release_year >= 2012)&(data.release_year <= 2014)]
answers[10] = data_select[data_select.profit==data_select.profit.min()].original_title.item()
answers[10]

'The Lone Ranger'

In [25]:
# Посмотрим самые убыточные фильмы за последние 10 лет
data.sort_values(by='profit') \
    .drop_duplicates(['release_year']) \
    .sort_values(by='release_year', ascending=False) \
    [['release_year', 'original_title', 'profit']].head(10)

,release_year,original_title,profit
53,2015,Blackhat,-52247060
144,2014,The Interview,-32694825
1245,2013,The Lone Ranger,-165710090
1007,2012,Upside Down,-51893525
809,2011,Mars Needs Moms,-111007242
468,2010,How Do You Know,-71331093
283,2009,Mr. Nobody,-43452791
636,2008,Australia,-80445998
1718,2007,Awake,-71626175
1464,2006,Flushed Away,-84540684


# 11. Какого жанра фильмов больше всего?

In [26]:
answers[11] = pd.Series(data.genres.str.cat(sep='|').split('|')).value_counts().index[0]
answers[11]

'Drama'

ВАРИАНТ 2

In [27]:
Counter(data.genres.str.cat(sep='|').split('|')).most_common()

[('Drama', 782),
 ('Comedy', 683),
 ('Thriller', 596),
 ('Action', 582),
 ('Adventure', 415),
 ('Crime', 315),
 ('Romance', 308),
 ('Family', 260),
 ('Science Fiction', 248),
 ('Fantasy', 222),
 ('Horror', 176),
 ('Mystery', 168),
 ('Animation', 139),
 ('Music', 64),
 ('History', 62),
 ('War', 58),
 ('Western', 19),
 ('Documentary', 8),
 ('Foreign', 2)]

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [28]:
answers[12] = data[data.profit>0].explode('list_genres').list_genres.value_counts().index[0]
answers[12]

'Drama'

In [29]:
# Посмотрим количество прибыльных фильмов по жанрам
data[data.profit>0].explode('list_genres').list_genres.value_counts()

Drama              560
Comedy             551
Thriller           446
Action             444
Adventure          337
Romance            242
Crime              231
Family             226
Science Fiction    195
Fantasy            188
Horror             150
Animation          120
Mystery            119
Music               47
History             46
War                 41
Western             12
Documentary          7
Name: list_genres, dtype: int64

# 13. У какого режиссера самые большие суммарные кассовые сбооры?

In [30]:
answers[13] = data.explode('list_director').groupby('list_director').revenue.sum() \
    .sort_values(ascending=False) \
    .index[0]
answers[13]

# Примечание: когда у одного фильма несколько режисеров, выручка в полном размере добавляется каждому

'Peter Jackson'

In [31]:
# Посмотрим список режисеров в порядке убывания выручки
data.explode('list_director').groupby('list_director').revenue.sum().sort_values(ascending=False)

list_director
Peter Jackson        6490593685
Christopher Nolan    4167548502
David Yates          4154295625
Michael Bay          3886938960
J.J. Abrams          3579169916
                        ...    
David MichÃ´d           2295423
Steven Shainberg        2281089
Paul Schrader           2062066
Keanu Reeves            2054941
Simon Hunter            2033165
Name: revenue, Length: 997, dtype: int64

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [32]:
answers[14] = data[data.genres.str.contains('Action')].explode('list_director').list_director.value_counts().index[0]
answers[14]

'Robert Rodriguez'

In [33]:
# Посмотрим список режисеров в порядке убывания количества фильмов в жанре Action
data[data.genres.str.contains('Action')].explode('list_director').list_director.value_counts()

Robert Rodriguez      9
Michael Bay           7
Paul W.S. Anderson    7
Ridley Scott          6
Antoine Fuqua         6
                     ..
Daniel Espinosa       1
Mikael HÃ¥fstrÃ¶m     1
Christian Duguay      1
Allen Hughes          1
Tamra Davis           1
Name: list_director, Length: 364, dtype: int64

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [34]:
answers[15] = data[data.release_year==2012].explode('list_cast').groupby('list_cast').revenue.sum() \
    .sort_values(ascending=False) \
    .index[0]
answers[15]

# Примечание: когда в одном фильме снимается несколько актеров, вся выручка добавляется каждому

'Chris Hemsworth'

In [35]:
# Посмотрим список актеров в порядке убывания кассовых сборов фильмов 2012 года с их участием
data[data.release_year==2012].explode('list_cast').groupby('list_cast').revenue.sum().sort_values(ascending=False)

list_cast
Chris Hemsworth      2027450773
Denis Leary          1629460639
Anne Hathaway        1522851057
Chris Evans          1519557910
Robert Downey Jr.    1519557910
                        ...    
Jason Bateman           3428048
Danny Huston            2106557
Sami Gayle              2106557
Josh Lucas              2106557
Nicolas Cage            2106557
Name: revenue, Length: 466, dtype: int64

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [36]:
answers[16] = data[data.budget>data.budget.mean()].explode('list_cast').list_cast.value_counts().index[0]
answers[16]

# Примечание: высокобюджетный - бюджет фильма выше среднего значения

'Matt Damon'

In [37]:
# Посмотрим список актеров в порядке убывания количества высокобююджетных фильмов с их участием
data[data.budget>data.budget.mean()].explode('list_cast').list_cast.value_counts()

Matt Damon          18
Adam Sandler        17
Angelina Jolie      16
Eddie Murphy        15
Tom Cruise          15
                    ..
Trevor Morgan        1
Heather Locklear     1
Rupert Evans         1
Richard Gant         1
Jordan Nagai         1
Name: list_cast, Length: 1505, dtype: int64

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [38]:
answers[17] = data[data.cast.str.contains('Nicolas Cage')].explode('list_genres').list_genres.value_counts().index[0]
answers[17]

'Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [39]:
data_select = data[data.production_companies.str.contains('Paramount Pictures')]
answers[18] = data_select[data_select.profit==data_select.profit.min()].original_title.item()
answers[18]

'K-19: The Widowmaker'

In [40]:
# Посмотрим топ10 самых убыточных фильмов от Paramount Pictures
data[data.production_companies.str.contains('Paramount Pictures')] \
    .sort_values(by=['profit'])[['original_title', 'profit']].head(10)

,original_title,profit
925,K-19: The Widowmaker,-64831034
1123,Timeline,-60519261
1722,Next,-51788987
1623,Alfie,-46604061
1631,Twisted,-24805000
587,Domestic Disturbance,-20750706
667,The Love Guru,-20180936
1059,A Thousand Words,-17955723
436,The Fighter,-15809131
959,Abandon,-14280643


# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [41]:
answers[19] = data.groupby(by='release_year').revenue.sum().sort_values(ascending=False).index[0]
answers[19]

2015

In [42]:
# Посмотрим суммарные кассовые сборы по годам
data.groupby(by='release_year').revenue.sum().sort_index()

release_year
2000    10664099805
2001    13017764865
2002    14136361487
2003    14346123312
2004    15663430720
2005    15309425558
2006    14775042320
2007    18162406801
2008    18252781990
2009    20261791024
2010    21071204961
2011    22676791872
2012    23079001687
2013    23213799791
2014    23405862953
2015    25449202382
Name: revenue, dtype: int64

# 20. Какой самый прибыльный год для студии Warner Bros?

In [43]:
answers[20] = data[data.production_companies.str.contains('Warner Bros')] \
    .groupby(by='release_year').profit.sum().sort_values(ascending=False).index[0]
answers[20]

2014

In [44]:
# Посмотрим прибыль студии Warner Bros по годам
data[data.production_companies.str.contains('Warner Bros')].groupby(by='release_year').profit.sum().sort_index()

release_year
2000     452631386
2001    1343545668
2002    1022709901
2003    1855493377
2004    1631933725
2005    1551980298
2006     620170743
2007    2201675217
2008    2134595031
2009    1822454136
2010    1974712985
2011    1871393682
2012    1258020056
2013    1636453400
2014    2295464519
2015     870368348
Name: profit, dtype: int64

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [45]:
answers[21] = data.release_month.value_counts().index[0]
answers[21]

9

In [46]:
# Посмотрим сколько фильмов выходило в разные месяцы
data.release_month.value_counts().sort_index()

1     110
2     135
3     156
4     149
5     140
6     147
7     142
8     161
9     227
10    186
11    146
12    190
Name: release_month, dtype: int64

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [47]:
answers[22] = sum(data.release_month.value_counts()[month] for month in [6,7,8])
answers[22]

450

In [48]:
# Вариант 2, после добавления в предобработке поля season
len(data[data.release_season=='summer'])

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [49]:
answers[23] = data[data.release_season=='winter'].explode('list_director').list_director.value_counts().index[0]
answers[23]

'Peter Jackson'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [50]:
# Дважды ошибся в ответе на этот вопрос

In [51]:
# Сначала указал студию, выпустившую наибольшее количество фильмов с названиями длиннее, чем в среднем
answers[24] = data[data.original_title.str.len()>data.original_title.str.len().mean()] \
    .explode('list_production_companies') \
    .list_production_companies.value_counts() \
    .index[0]
answers[24]

'Universal Pictures'

In [52]:
# Во второй раз указал студию, названия фильмов которой суммарно длиннее, чем у других студий
answers[24] = data[['original_title', 'list_production_companies']] \
    .assign(original_title_length = data.original_title.str.len()) \
    .explode('list_production_companies') \
    .groupby(by='list_production_companies').sum().original_title_length \
    .sort_values(ascending=False) \
    .index[0]
answers[24]

'Warner Bros.'

In [53]:
# Предполагаю, что надо было указать студию с максимальной средней длиной названий фильмов
answers[24] = data[['original_title', 'list_production_companies']] \
    .assign(original_title_length = data.original_title.str.len()) \
    .explode('list_production_companies') \
    .groupby(by='list_production_companies').mean().original_title_length \
    .sort_values(ascending=False) \
    .index[0]
answers[24]

'Four By Two Productions'

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [54]:
answers[25] = data[['original_title', 'overview', 'list_production_companies']] \
    .assign(overview_worlds_count = data.overview.str.split().str.len()) \
    .explode('list_production_companies') \
    .groupby(by='list_production_companies').overview_worlds_count.mean() \
    .sort_values(ascending=False) \
    .index[0]
answers[25]

'Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [55]:
# Определим список фильмов, имеющих рейтинг, позволяющий войти в 1% лучших
answers[26] = sorted(data[data.vote_average >= data.describe(percentiles=[0.99]).vote_average['99%']].original_title)
answers[26]

['12 Years a Slave',
 '3 Idiots',
 'Big Hero 6',
 'Dallas Buyers Club',
 'Eternal Sunshine of the Spotless Mind',
 'Gone Girl',
 'Guardians of the Galaxy',
 'Her',
 'Inception',
 'Inside Out',
 'Interstellar',
 'Memento',
 'Mr. Nobody',
 'Prisoners',
 'Room',
 'Spotlight',
 'The Dark Knight',
 'The Fault in Our Stars',
 'The Grand Budapest Hotel',
 'The Imitation Game',
 'The Lord of the Rings: The Fellowship of the Ring',
 'The Lord of the Rings: The Return of the King',
 'The Lord of the Rings: The Two Towers',
 'The Pianist',
 'The Prestige',
 'The Theory of Everything',
 'The Wolf of Wall Street',
 'There Will Be Blood']

In [56]:
# Вариант 2, с самостоятельным расчетом граничного рейтинга, в первом варианте использована встроенная функция
rating_top1_percent = data.sort_values(by='vote_average', ascending=False).vote_average.iloc[int(len(data)*0.01)]
sorted(data[data.vote_average >= rating_top1_percent].original_title)

['12 Years a Slave',
 '3 Idiots',
 'Big Hero 6',
 'Dallas Buyers Club',
 'Eternal Sunshine of the Spotless Mind',
 'Gone Girl',
 'Guardians of the Galaxy',
 'Her',
 'Inception',
 'Inside Out',
 'Interstellar',
 'Memento',
 'Mr. Nobody',
 'Prisoners',
 'Room',
 'Spotlight',
 'The Dark Knight',
 'The Fault in Our Stars',
 'The Grand Budapest Hotel',
 'The Imitation Game',
 'The Lord of the Rings: The Fellowship of the Ring',
 'The Lord of the Rings: The Return of the King',
 'The Lord of the Rings: The Two Towers',
 'The Pianist',
 'The Prestige',
 'The Theory of Everything',
 'The Wolf of Wall Street',
 'There Will Be Blood']

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [57]:
Counter(chain.from_iterable(data.list_cast.apply(combinations, r=2))).most_common(5)

[(('Daniel Radcliffe', 'Rupert Grint'), 8),
 (('Daniel Radcliffe', 'Emma Watson'), 8),
 (('Rupert Grint', 'Emma Watson'), 7),
 (('Ben Stiller', 'Owen Wilson'), 6),
 (('Johnny Depp', 'Helena Bonham Carter'), 6)]

In [58]:
# Подготовим ответ на вопрос. Это не так просто оказалось, учитывая несколько вариантов ответа
pairs = pd.Series(Counter(chain.from_iterable(data.list_cast.apply(combinations, r=2))))
answers[27] = list(pairs[pairs == pairs.max()].index)
answers[27]

[('Daniel Radcliffe', 'Rupert Grint'), ('Daniel Radcliffe', 'Emma Watson')]

In [59]:
# Вариант 2, расчет с использованием цикла
pairs = Counter()
for cast in data.list_cast:
    # Создаем пары для всех актеров в списке и идем по ним
    for pair in list(combinations(cast, 2)):
        if pair not in pairs:
            pairs[pair] = 1
        else:
            pairs[pair] += 1
pairs.most_common(5)

[(('Daniel Radcliffe', 'Rupert Grint'), 8),
 (('Daniel Radcliffe', 'Emma Watson'), 8),
 (('Rupert Grint', 'Emma Watson'), 7),
 (('Ben Stiller', 'Owen Wilson'), 6),
 (('Johnny Depp', 'Helena Bonham Carter'), 6)]

In [60]:
# Сравним варианты по времени

In [61]:
%%timeit
Counter(chain.from_iterable(data.list_cast.apply(combinations, r=2))).most_common(5)

6.86 ms ± 662 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [62]:
%%timeit
pairs = Counter()
for cast in data.list_cast:
    # Создаем пары для всех актеров в списке и идем по ним
    for pair in list(combinations(cast, 2)):
        if pair not in pairs:
            pairs[pair] = 1
        else:
            pairs[pair] += 1
pairs.most_common(5)

9.3 ms ± 278 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Submission

In [63]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{1: 'Pirates of the Caribbean: On Stranger Tides',
 2: 'Gods and Generals',
 3: 'Winnie the Pooh',
 4: 110,
 5: 107.0,
 6: 'Avatar',
 7: 'The Lone Ranger',
 8: 1478,
 9: 'The Dark Knight',
 10: 'The Lone Ranger',
 11: 'Drama',
 12: 'Drama',
 13: 'Peter Jackson',
 14: 'Robert Rodriguez',
 15: 'Chris Hemsworth',
 16: 'Matt Damon',
 17: 'Action',
 18: 'K-19: The Widowmaker',
 19: 2015,
 20: 2014,
 21: 9,
 22: 450,
 23: 'Peter Jackson',
 24: 'Four By Two Productions',
 25: 'Midnight Picture Show',
 26: ['12 Years a Slave',
  '3 Idiots',
  'Big Hero 6',
  'Dallas Buyers Club',
  'Eternal Sunshine of the Spotless Mind',
  'Gone Girl',
  'Guardians of the Galaxy',
  'Her',
  'Inception',
  'Inside Out',
  'Interstellar',
  'Memento',
  'Mr. Nobody',
  'Prisoners',
  'Room',
  'Spotlight',
  'The Dark Knight',
  'The Fault in Our Stars',
  'The Grand Budapest Hotel',
  'The Imitation Game',
  'The Lord of the Rings: The Fellowship of the Ring',
  'The Lord of the Rings: The Return of the King'

In [64]:
# и убедиться что ни чего не пропустил)
len(answers)

27